In [12]:
import pandas as pd

data = df = pd.read_parquet('/root/autodl-tmp/WSDM/input/train.parquet')

In [2]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    data.loc[val_idx, 'fold'] = int(fold)
data['fold'] = data['fold'].astype(int)

In [3]:
data.to_csv('/root/autodl-tmp/WSDM/input/train.csv', index=False)

In [1]:
import pandas as pd

data = pd.read_csv('/root/autodl-tmp/WSDM/submission.csv')
data2 = pd.read_parquet('/root/autodl-tmp/WSDM/hf-open-models-v1.parquet')

In [2]:
data = pd.merge(data, data2, on='id', how='left')

In [3]:
import numpy as np
filter = data[(data['model_a_prob'] > 0.5) | (data['model_b_prob'] > 0.5)]
filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')

/tmp/ipykernel_56494/4221851404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')


In [4]:
filter[['id', 'prompt',	'response_a','response_b','winner',	'model_a',	'model_b',	'language']].to_csv('/root/autodl-tmp/WSDM/input/extra.csv', index=False)

In [5]:
filter.shape

(8417, 10)

: 